# 🤖 Modelos de Machine Learning para RGM

## Objetivo
Implementar modelos de ML y Deep Learning para:
- **Elasticidad de Precios**: Análisis de sensibilidad precio-demanda
- **Segmentación Avanzada**: Clustering inteligente de clientes
- **Predicción de CLV**: Modelos predictivos de valor de cliente
- **Optimización de Promociones**: ML para ROI promocional
- **Predicción de Demanda**: Deep Learning para forecasting
- **Recomendación de Precios**: Neural Networks para pricing óptimo

## Metodología RGM + ML
1. **Price Elasticity Modeling** - Regresión y modelos econométricos
2. **Customer Segmentation** - K-means, DBSCAN, Gaussian Mixture
3. **CLV Prediction** - Random Forest, XGBoost, Neural Networks
4. **Promotion Optimization** - Reinforcement Learning
5. **Demand Forecasting** - LSTM, GRU, Transformer
6. **Dynamic Pricing** - Deep Q-Learning

## 1. Importación de Librerías

In [13]:
# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture


#from sklearn.cluster import KMeans, DBSCAN, GaussianMixture
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.decomposition import PCA
import xgboost as xgb
import lightgbm as lgb

# Deep Learning
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential, Model
    from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, BatchNormalization
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    print('✅ TensorFlow/Keras disponible')
except ImportError:
    print('⚠️ TensorFlow no disponible - usando solo sklearn')

# Estadística y econometría
try:
    import statsmodels.api as sm
    from statsmodels.tsa.seasonal import seasonal_decompose
    print('✅ Statsmodels disponible')
except ImportError:
    print('⚠️ Statsmodels no disponible')

print('🚀 Librerías ML/DL cargadas exitosamente')

✅ TensorFlow/Keras disponible
✅ Statsmodels disponible
🚀 Librerías ML/DL cargadas exitosamente


## 2. Carga y Preparación de Datos

In [14]:
# Definir rutas
DATA_PATH = '../data/processed/'

# Función para limpiar formato numérico con comas decimales
def clean_numeric_columns(df, numeric_cols):
    """
    Convierte columnas numéricas que usan coma como separador decimal
    """
    df_clean = df.copy()
    for col in numeric_cols:
        if col in df_clean.columns:
            # Convertir a string primero para manejar valores no string
            df_clean[col] = df_clean[col].astype(str)
            # Reemplazar comas por puntos decimales
            df_clean[col] = df_clean[col].str.replace(',', '.', regex=False)
            # Manejar valores vacíos o no numéricos
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    return df_clean

# Cargar datasets
print('📊 Cargando datasets para ML...')

# Cargar datos principales
df_ventas = pd.read_csv(DATA_PATH + 'datos_limpios.csv')

# Identificar columnas numéricas que pueden tener formato con comas
numeric_columns = ['PRECIO', 'QTY_ENTREGADA', 'QTY_PEDIDA', 'QTY_SUGERIDA', 'QTY_KILOS', 
                  'IMPORTE', 'IMPORTE_DSCTO', 'PRECIO_COSTOSTD', 'PCTJ_DSCTO']

# Limpiar formato numérico
print('🔄 Limpiando formato numérico...')
df_ventas = clean_numeric_columns(df_ventas, numeric_columns)

# Cargar otros datasets
df_clientes = pd.read_csv(DATA_PATH + 'maestro_clientes_rgm.csv')
df_productos = pd.read_csv(DATA_PATH + 'maestro_productos_rgm.csv')
df_promociones = pd.read_csv(DATA_PATH + 'analisis_promociones_rgm.csv')

print(f'Transacciones: {df_ventas.shape[0]:,}')
print(f'Clientes: {df_clientes.shape[0]:,}')
print(f'Productos: {df_productos.shape[0]:,}')
print(f'Promociones: {df_promociones.shape[0]:,}')

# Verificar columnas de datos principales
print('\n📋 Columnas disponibles:')
print('Ventas:', list(df_ventas.columns))
print('Clientes:', list(df_clientes.columns))
print('Productos:', list(df_productos.columns))

# Verificar que las columnas numéricas se cargaron correctamente
print('\n🔍 Verificación de datos numéricos:')
for col in ['PRECIO', 'QTY_ENTREGADA', 'IMPORTE']:
    if col in df_ventas.columns:
        print(f'{col}: min={df_ventas[col].min()}, max={df_ventas[col].max()}, nulls={df_ventas[col].isnull().sum()}')

📊 Cargando datasets para ML...
🔄 Limpiando formato numérico...
Transacciones: 4,090,711
Clientes: 21,269
Productos: 1,600
Promociones: 2,908

📋 Columnas disponibles:
Ventas: ['ï»¿"ORDEN"', 'POSICION', 'SECUENCIA', 'ID_CLIENTE', 'ARTICULO', 'CNTR', 'SKU', 'ID_ALMACEN', 'TIPO_TRANSACCION', 'NRO_FACTURA', 'FECHA_ORDEN_FECHA', 'FECHA_ORDEN_HORA', 'FECHA_ENTREGA_FECHA', 'FECHA_ENTREGA_HORA', 'FECHA_FACTURA_FECHA', 'FECHA_FACTURA_HORA', 'QTY_PEDIDA', 'QTY_ENTREGADA', 'QTY_SUGERIDA', 'QTY_RETRO_ORDEN', 'STATUS_VTA', 'UND_PRECIO_VTA', 'FACTOR_PRECIO_UND_VTA', 'ID_FAMILIA_ORDEN', 'UND_VENTA', 'FACTOR_UND_VTA', 'ID_CONDICIONPAGO', 'ID_TIPOORDEN', 'TIPO_ORDEN', 'ID_VENDEDOR', 'ID_CATEGORIA_VDI', 'CLIENTE_ANEXO', 'SOURCE_TBL', 'ANIO', 'PERIODO', 'ID_PROMOCION', 'NOMBRE_PROMOCION', 'CTA_CTBLE', 'PEDIDO_MTO', 'TIPO_LINEA', 'UND_X_CAJA', 'ID_UNDSTOCK', 'QTY_KILOS', 'PRECIO_COSTOSTD', 'PRECIO', 'PCTJ_DSCTO', 'IMPORTE_DSCTO', 'IMPORTE', 'IMPORTE_DSCTO_LINEA_ORDEN', 'IMPORTE_DSCTO_ORDEN', 'VALOR_DSCTO_A

## 3. Modelo de Elasticidad de Precios 📈

### Análisis econométrico de la relación precio-demanda

In [17]:
print('📈 ANÁLISIS DE ELASTICIDAD DE PRECIOS')
print('=' * 50)

# Preparar datos para elasticidad
# Necesitamos: SKU, Precio, Cantidad, Período

# Crear dataset agregado por producto-período si no existe fecha en ventas
if 'Fecha' in df_ventas.columns:
    df_ventas['Fecha'] = pd.to_datetime(df_ventas['Fecha'])
    df_ventas['Periodo'] = df_ventas['Fecha'].dt.to_period('M')
    
    # Agregar por SKU-Período
    elasticity_data = df_ventas.groupby(['SKU', 'Periodo']).agg({
        'PRECIO': 'mean',
        'QTY_ENTREGADA': 'sum',
        'IMPORTE': 'sum'
    }).reset_index()
else:
    # Si no hay fecha, usar productos con mayor variación de precios
    price_variation = df_ventas.groupby('SKU')['PRECIO'].agg(['std', 'count', 'mean']).reset_index()
    price_variation = price_variation[price_variation['count'] >= 100]  # Productos con suficientes observaciones
    high_variation_skus = price_variation.nlargest(50, 'std')['SKU'].tolist()
    
    # Crear bins de precios para simular períodos
    elasticity_data = df_ventas[df_ventas['SKU'].isin(high_variation_skus)].copy()
    
    # Crear quintiles de precios por SKU
    elasticity_data['Precio_Quintil'] = elasticity_data.groupby('SKU')['PRECIO'].transform(
        lambda x: pd.qcut(x, q=5, labels=False, duplicates='drop')
    )
    
    # Agregar por SKU-Quintil
    elasticity_data = elasticity_data.groupby(['SKU', 'Precio_Quintil']).agg({
        'PRECIO': 'mean',
        'QTY_ENTREGADA': 'sum',
        'IMPORTE': 'sum'
    }).reset_index()

print(f'Dataset para elasticidad: {elasticity_data.shape[0]:,} observaciones')
print(f'Productos únicos: {elasticity_data["SKU"].nunique():,}')

# Mostrar muestra
elasticity_data.head()

📈 ANÁLISIS DE ELASTICIDAD DE PRECIOS
Dataset para elasticidad: 57 observaciones
Productos únicos: 50


,SKU,Precio_Quintil,PRECIO,QTY_ENTREGADA,IMPORTE
0,5063031/Q13,0,47.490000,927.0,6971.92
1,5063032/Q14,0,45.449916,1213.0,7364.71
2,5063033/Q15,0,64.660220,29804.0,146408.92
3,5063033/Q18,0,61.838018,583.0,9173.44
4,5063034/Q16,0,83.996488,19506.0,108886.06


In [19]:
# Calcular elasticidad precio-demanda por producto
def calculate_price_elasticity(df, sku_col='SKU', price_col='PRECIO', qty_col='QTY_ENTREGADA'):
    """
    Calcula elasticidad precio-demanda usando regresión log-log
    Elasticidad = % cambio en cantidad / % cambio en precio
    """
    if len(df) == 0:
        print('⚠️ DataFrame vacío para cálculo de elasticidad')
        return pd.DataFrame()
    
    elasticities = []
    
    for sku in df[sku_col].unique():
        sku_data = df[df[sku_col] == sku].copy()
        
        if len(sku_data) < 3:  # Necesitamos al menos 3 puntos
            continue
            
        # Filtrar valores válidos y únicos
        sku_data = sku_data[
            (sku_data[price_col] > 0) & 
            (sku_data[qty_col] > 0) &
            sku_data[price_col].notna() & 
            sku_data[qty_col].notna()
        ].drop_duplicates(subset=[price_col, qty_col])
        
        if len(sku_data) < 3:
            continue
            
        # Verificar que hay variación en precios
        if sku_data[price_col].std() == 0:
            continue
            
        try:
            # Transformación log-log
            log_price = np.log(sku_data[price_col])
            log_qty = np.log(sku_data[qty_col])
            
            # Verificar que los logs son válidos
            if log_price.isna().any() or log_qty.isna().any():
                continue
                
            # Regresión linear en logs
            X = log_price.values.reshape(-1, 1)
            y = log_qty.values
            
            model = LinearRegression().fit(X, y)
            elasticity = model.coef_[0]  # Coeficiente = elasticidad
            r2 = model.score(X, y)
            
            elasticities.append({
                'SKU': sku,
                'Elasticidad': elasticity,
                'R2': r2,
                'Observaciones': len(sku_data),
                'Precio_Promedio': sku_data[price_col].mean(),
                'Cantidad_Promedio': sku_data[qty_col].mean(),
                'Precio_Min': sku_data[price_col].min(),
                'Precio_Max': sku_data[price_col].max()
            })
        except Exception as e:
            print(f'Error procesando SKU {sku}: {str(e)}')
            continue
    
    return pd.DataFrame(elasticities)

# Calcular elasticidades solo si hay datos
if len(elasticity_data) > 0:
    print('🔄 Calculando elasticidades precio-demanda...')
    elasticity_results = calculate_price_elasticity(elasticity_data)
    
    if len(elasticity_results) > 0:
        # Filtrar resultados válidos
        elasticity_results = elasticity_results[
            (elasticity_results['R2'] > 0.1) &  # R² mínimo
            (elasticity_results['Elasticidad'] < 0) &  # Elasticidad negativa (normal)
            (elasticity_results['Elasticidad'] > -10) &  # Elasticidad razonable
            (elasticity_results['Observaciones'] >= 3)  # Suficientes observaciones
        ]
        
        print(f'Productos con elasticidad válida: {len(elasticity_results)}')
        if len(elasticity_results) > 0:
            print(f'Elasticidad promedio: {elasticity_results["Elasticidad"].mean():.3f}')
            print(f'R² promedio: {elasticity_results["R2"].mean():.3f}')
            
            # Mostrar distribución
            display(elasticity_results.describe())
        else:
            print('❌ No se encontraron productos con elasticidad válida después del filtrado')
    else:
        print('❌ No se pudieron calcular elasticidades')
else:
    print('⚠️ Saltando cálculo de elasticidad - no hay datos disponibles')
    elasticity_results = pd.DataFrame()

🔄 Calculando elasticidades precio-demanda...
❌ No se pudieron calcular elasticidades


In [ ]:
# Visualización de elasticidades
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('📈 Análisis de Elasticidad Precio-Demanda', fontsize=16, fontweight='bold')

# 1. Distribución de elasticidades
axes[0,0].hist(elasticity_results['Elasticidad'], bins=30, alpha=0.7, color='steelblue')
axes[0,0].axvline(elasticity_results['Elasticidad'].mean(), color='red', linestyle='--', 
                  label=f'Media: {elasticity_results["Elasticidad"].mean():.3f}')
axes[0,0].axvline(-1, color='orange', linestyle='--', label='Elasticidad Unitaria')
axes[0,0].set_title('Distribución de Elasticidades')
axes[0,0].set_xlabel('Elasticidad Precio-Demanda')
axes[0,0].legend()

# 2. R² vs Elasticidad
scatter = axes[0,1].scatter(elasticity_results['Elasticidad'], elasticity_results['R2'], 
                           alpha=0.6, c=elasticity_results['Precio_Promedio'], cmap='viridis')
axes[0,1].set_title('R² vs Elasticidad')
axes[0,1].set_xlabel('Elasticidad')
axes[0,1].set_ylabel('R²')
plt.colorbar(scatter, ax=axes[0,1], label='Precio Promedio')

# 3. Categorización por elasticidad
def categorize_elasticity(elasticity):
    if elasticity > -0.5:
        return 'Inelástico (|E| < 0.5)'
    elif elasticity > -1:
        return 'Moderadamente Elástico (0.5 < |E| < 1)'
    elif elasticity > -2:
        return 'Elástico (1 < |E| < 2)'
    else:
        return 'Muy Elástico (|E| > 2)'

elasticity_results['Categoria'] = elasticity_results['Elasticidad'].apply(categorize_elasticity)
cat_counts = elasticity_results['Categoria'].value_counts()

axes[1,0].pie(cat_counts.values, labels=cat_counts.index, autopct='%1.1f%%', startangle=90)
axes[1,0].set_title('Categorización por Elasticidad')

# 4. Top 10 productos más/menos elásticos
top_elastic = elasticity_results.nsmallest(5, 'Elasticidad')
least_elastic = elasticity_results.nlargest(5, 'Elasticidad')

y_pos = np.arange(10)
products = list(least_elastic['SKU'].astype(str)) + list(top_elastic['SKU'].astype(str))
elasticities = list(least_elastic['Elasticidad']) + list(top_elastic['Elasticidad'])
colors = ['lightcoral']*5 + ['lightblue']*5

axes[1,1].barh(y_pos, elasticities, color=colors)
axes[1,1].set_yticks(y_pos)
axes[1,1].set_yticklabels([f'SKU {p[:8]}...' for p in products])
axes[1,1].set_title('Top 5 Menos/Más Elásticos')
axes[1,1].set_xlabel('Elasticidad')

plt.tight_layout()
plt.show()

# Imprimir insights
print('\n🎯 INSIGHTS DE ELASTICIDAD:')
print('=' * 40)
inelastic_pct = (elasticity_results['Elasticidad'] > -1).mean() * 100
print(f'• {inelastic_pct:.1f}% de productos son relativamente inelásticos (|E| < 1)')
print(f'• Producto más elástico: SKU {elasticity_results.loc[elasticity_results["Elasticidad"].idxmin(), "SKU"]} (E = {elasticity_results["Elasticidad"].min():.3f})')
print(f'• Producto menos elástico: SKU {elasticity_results.loc[elasticity_results["Elasticidad"].idxmax(), "SKU"]} (E = {elasticity_results["Elasticidad"].max():.3f})')

## 4. Segmentación Avanzada con ML 🎯

### Clustering inteligente de clientes usando múltiples algoritmos

In [ ]:
print('🎯 SEGMENTACIÓN AVANZADA CON MACHINE LEARNING')
print('=' * 55)

# Preparar features para clustering
clustering_features = [
    'Venta_Total', 'Ticket_Promedio', 'Num_Compras', 
    'Cantidad_Total', 'CLV_Estimado'
]

# Limpiar datos y manejar outliers
df_clustering = df_clientes[clustering_features].copy()

# Remover outliers usando IQR
def remove_outliers_iqr(df, columns):
    df_clean = df.copy()
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    return df_clean

# Aplicar limpieza de outliers
df_clustering_clean = remove_outliers_iqr(df_clustering, clustering_features)
print(f'Clientes después de remover outliers: {len(df_clustering_clean):,} ({len(df_clustering_clean)/len(df_clustering)*100:.1f}%)')

# Escalado de features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clustering_clean)

# Reducción de dimensionalidad para visualización
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print(f'Varianza explicada por PCA: {pca.explained_variance_ratio_.sum():.3f}')
print(f'Features escalados: {X_scaled.shape}')

In [ ]:
# Comparar múltiples algoritmos de clustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score

def evaluate_clustering(X, labels, algorithm_name):
    """
    Evalúa calidad del clustering usando múltiples métricas
    """
    if len(np.unique(labels)) < 2:
        return None
    
    silhouette = silhouette_score(X, labels)
    calinski = calinski_harabasz_score(X, labels)
    
    return {
        'Algorithm': algorithm_name,
        'Silhouette_Score': silhouette,
        'Calinski_Harabasz_Score': calinski,
        'N_Clusters': len(np.unique(labels)),
        'N_Noise': np.sum(labels == -1) if -1 in labels else 0
    }

# Probar diferentes algoritmos
clustering_results = []

# 1. K-Means con diferentes k
print('🔄 Probando K-Means...')
for k in range(3, 8):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    result = evaluate_clustering(X_scaled, labels, f'K-Means (k={k})')
    if result:
        clustering_results.append(result)

# 2. Gaussian Mixture Model
print('🔄 Probando Gaussian Mixture...')
for k in range(3, 8):
    gmm = GaussianMixture(n_components=k, random_state=42)
    labels = gmm.fit_predict(X_scaled)
    result = evaluate_clustering(X_scaled, labels, f'GMM (k={k})')
    if result:
        clustering_results.append(result)

# 3. DBSCAN con diferentes parámetros
print('🔄 Probando DBSCAN...')
for eps in [0.5, 0.8, 1.0, 1.2]:
    for min_samples in [10, 20]:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(X_scaled)
        result = evaluate_clustering(X_scaled, labels, f'DBSCAN (eps={eps}, min={min_samples})')
        if result and result['N_Clusters'] > 1:
            clustering_results.append(result)

# Convertir a DataFrame y mostrar resultados
results_df = pd.DataFrame(clustering_results)
results_df = results_df.sort_values('Silhouette_Score', ascending=False)

print('\n📊 RESULTADOS DE CLUSTERING:')
print('=' * 60)
print(results_df.head(10).to_string(index=False))

# Seleccionar mejor modelo
best_model = results_df.iloc[0]
print(f'\n🏆 Mejor modelo: {best_model["Algorithm"]}')
print(f'   Silhouette Score: {best_model["Silhouette_Score"]:.3f}')
print(f'   Clusters: {int(best_model["N_Clusters"])}')

In [ ]:
# Implementar el mejor modelo de clustering
best_algorithm = best_model['Algorithm']

if 'K-Means' in best_algorithm:
    k = int(best_algorithm.split('k=')[1].split(')')[0])
    final_model = KMeans(n_clusters=k, random_state=42, n_init=10)
elif 'GMM' in best_algorithm:
    k = int(best_algorithm.split('k=')[1].split(')')[0])
    final_model = GaussianMixture(n_components=k, random_state=42)
else:  # DBSCAN
    eps = float(best_algorithm.split('eps=')[1].split(',')[0])
    min_samples = int(best_algorithm.split('min=')[1].split(')')[0])
    final_model = DBSCAN(eps=eps, min_samples=min_samples)

# Ajustar modelo final
final_labels = final_model.fit_predict(X_scaled)

# Asignar clusters a datos originales
df_clustering_clean['Cluster_ML'] = final_labels

# Analizar características de cada cluster
cluster_analysis = df_clustering_clean.groupby('Cluster_ML')[clustering_features].agg([
    'count', 'mean', 'median', 'std'
]).round(2)

print('\n📋 ANÁLISIS POR CLUSTER:')
print('=' * 50)
for cluster in sorted(df_clustering_clean['Cluster_ML'].unique()):
    if cluster != -1:  # Ignorar ruido en DBSCAN
        cluster_data = df_clustering_clean[df_clustering_clean['Cluster_ML'] == cluster]
        size = len(cluster_data)
        pct = size / len(df_clustering_clean) * 100
        avg_clv = cluster_data['CLV_Estimado'].mean()
        avg_ticket = cluster_data['Ticket_Promedio'].mean()
        avg_compras = cluster_data['Num_Compras'].mean()
        
        print(f'\nCluster {cluster}:')
        print(f'  • Tamaño: {size:,} clientes ({pct:.1f}%)')
        print(f'  • CLV Promedio: ${avg_clv:,.0f}')
        print(f'  • Ticket Promedio: ${avg_ticket:.2f}')
        print(f'  • Compras Promedio: {avg_compras:.0f}')

# Comparar con segmentación original
# Crear mapeo para comparar
df_comparison = df_clientes.copy()
df_comparison['Cluster_ML'] = -1  # Inicializar

# Asignar clusters solo a los datos limpios
clean_indices = df_clustering_clean.index
df_comparison.loc[clean_indices, 'Cluster_ML'] = final_labels

# Tabla de contingencia
if 'Segmento' in df_comparison.columns:
    contingency = pd.crosstab(df_comparison['Segmento'], df_comparison['Cluster_ML'], 
                             margins=True, normalize='index') * 100
    print('\n🔄 COMPARACIÓN SEGMENTACIÓN ORIGINAL vs ML:')
    print('=' * 55)
    print(contingency.round(1))

In [ ]:
# Visualización de clusters
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('🎯 Segmentación Avanzada con Machine Learning', fontsize=16, fontweight='bold')

# 1. Clusters en espacio PCA
unique_clusters = sorted([c for c in np.unique(final_labels) if c != -1])
colors = plt.cm.Set1(np.linspace(0, 1, len(unique_clusters)))

for i, cluster in enumerate(unique_clusters):
    cluster_mask = final_labels == cluster
    axes[0,0].scatter(X_pca[cluster_mask, 0], X_pca[cluster_mask, 1], 
                     c=[colors[i]], label=f'Cluster {cluster}', alpha=0.6)

if -1 in final_labels:  # Ruido en DBSCAN
    noise_mask = final_labels == -1
    axes[0,0].scatter(X_pca[noise_mask, 0], X_pca[noise_mask, 1], 
                     c='black', label='Ruido', alpha=0.3, s=10)

axes[0,0].set_title('Clusters en Espacio PCA')
axes[0,0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} var)')
axes[0,0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} var)')
axes[0,0].legend()

# 2. Distribución de tamaños de clusters
cluster_sizes = pd.Series(final_labels).value_counts().sort_index()
if -1 in cluster_sizes.index:
    cluster_sizes = cluster_sizes.drop(-1)  # Remover ruido

axes[0,1].bar(range(len(cluster_sizes)), cluster_sizes.values, color=colors[:len(cluster_sizes)])
axes[0,1].set_title('Distribución de Clusters')
axes[0,1].set_xlabel('Cluster')
axes[0,1].set_ylabel('Número de Clientes')
axes[0,1].set_xticks(range(len(cluster_sizes)))
axes[0,1].set_xticklabels([f'C{i}' for i in cluster_sizes.index])

# 3. CLV por cluster
clv_by_cluster = []
cluster_labels = []
for cluster in unique_clusters:
    cluster_data = df_clustering_clean[df_clustering_clean['Cluster_ML'] == cluster]['CLV_Estimado']
    clv_by_cluster.append(cluster_data)
    cluster_labels.append(f'C{cluster}')

axes[1,0].boxplot(clv_by_cluster, labels=cluster_labels)
axes[1,0].set_title('CLV por Cluster')
axes[1,0].set_ylabel('CLV Estimado')
axes[1,0].tick_params(axis='x', rotation=45)

# 4. Heatmap de características promedio por cluster
cluster_means = df_clustering_clean.groupby('Cluster_ML')[clustering_features].mean()
if -1 in cluster_means.index:
    cluster_means = cluster_means.drop(-1)

# Normalizar para heatmap
cluster_means_norm = (cluster_means - cluster_means.min()) / (cluster_means.max() - cluster_means.min())

im = axes[1,1].imshow(cluster_means_norm.T, cmap='YlOrRd', aspect='auto')
axes[1,1].set_title('Perfil de Clusters (Normalizado)')
axes[1,1].set_xticks(range(len(cluster_means_norm)))
axes[1,1].set_xticklabels([f'C{i}' for i in cluster_means_norm.index])
axes[1,1].set_yticks(range(len(clustering_features)))
axes[1,1].set_yticklabels([f.replace('_', '\n') for f in clustering_features])

# Colorbar
plt.colorbar(im, ax=axes[1,1], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

## 5. Predicción de CLV con ML/DL 💰

### Modelos predictivos para valor de vida del cliente

In [ ]:
print('💰 PREDICCIÓN DE CLV CON MACHINE LEARNING')
print('=' * 50)

# Preparar features para predicción de CLV
clv_features = [
    'Venta_Total', 'Ticket_Promedio', 'Num_Compras', 'Cantidad_Total'
]

# Dataset para CLV
df_clv = df_clientes[clv_features + ['CLV_Estimado']].copy()
df_clv = df_clv.dropna()

# Crear features adicionales
df_clv['Compra_Frecuencia'] = df_clv['Num_Compras'] / df_clv['Venta_Total']  # Compras por peso de venta
df_clv['Cantidad_por_Compra'] = df_clv['Cantidad_Total'] / df_clv['Num_Compras']
df_clv['Venta_por_Cantidad'] = df_clv['Venta_Total'] / df_clv['Cantidad_Total']

# Limpiar infinitos y NaN
df_clv = df_clv.replace([np.inf, -np.inf], np.nan).dropna()

# Features finales
feature_cols = clv_features + ['Compra_Frecuencia', 'Cantidad_por_Compra', 'Venta_por_Cantidad']
X = df_clv[feature_cols]
y = df_clv['CLV_Estimado']

print(f'Dataset CLV: {X.shape[0]:,} clientes, {X.shape[1]} features')
print(f'CLV promedio: ${y.mean():,.0f}')
print(f'CLV mediano: ${y.median():,.0f}')

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalado
scaler_clv = StandardScaler()
X_train_scaled = scaler_clv.fit_transform(X_train)
X_test_scaled = scaler_clv.transform(X_test)

print(f'Train: {X_train.shape[0]:,} | Test: {X_test.shape[0]:,}')

In [ ]:
# Probar múltiples modelos de ML
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """
    Evalúa modelo y retorna métricas
    """
    # Entrenar
    model.fit(X_train, y_train)
    
    # Predicciones
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Métricas
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    test_mse = mean_squared_error(y_test, y_pred_test)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    
    return {
        'Model': model_name,
        'Train_R2': train_r2,
        'Test_R2': test_r2,
        'Test_RMSE': np.sqrt(test_mse),
        'Test_MAE': test_mae,
        'Overfitting': train_r2 - test_r2,
        'Predictions': y_pred_test
    }

# Definir modelos
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=1.0),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42),
    'LightGBM': lgb.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
}

# Evaluar modelos
print('🔄 Entrenando modelos de ML...')
ml_results = []

for name, model in models.items():
    try:
        # Usar datos escalados para modelos lineales
        if 'Ridge' in name or 'Lasso' in name:
            result = evaluate_model(model, X_train_scaled, X_test_scaled, y_train, y_test, name)
        else:
            result = evaluate_model(model, X_train, X_test, y_train, y_test, name)
        
        ml_results.append(result)
        print(f'✅ {name}: R² = {result["Test_R2"]:.3f}')
    except Exception as e:
        print(f'❌ Error con {name}: {str(e)}')

# Convertir a DataFrame
results_clv_df = pd.DataFrame([{k:v for k,v in r.items() if k != 'Predictions'} for r in ml_results])
results_clv_df = results_clv_df.sort_values('Test_R2', ascending=False)

print('\n📊 RESULTADOS MODELOS CLV:')
print('=' * 70)
print(results_clv_df.round(4).to_string(index=False))

# Mejor modelo
best_clv_model = results_clv_df.iloc[0]
print(f'\n🏆 Mejor modelo: {best_clv_model["Model"]}')
print(f'   Test R²: {best_clv_model["Test_R2"]:.3f}')
print(f'   RMSE: ${best_clv_model["Test_RMSE"]:,.0f}')
print(f'   MAE: ${best_clv_model["Test_MAE"]:,.0f}')

In [ ]:
# Deep Learning para CLV (si TensorFlow está disponible)
if 'tensorflow' in locals():
    print('🧠 DEEP LEARNING PARA CLV')
    print('=' * 30)
    
    # Arquitectura de red neuronal
    def create_clv_model(input_dim):
        model = Sequential([
            Dense(128, activation='relu', input_shape=(input_dim,)),
            BatchNormalization(),
            Dropout(0.3),
            
            Dense(64, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            
            Dense(32, activation='relu'),
            Dropout(0.1),
            
            Dense(1, activation='linear')  # Regresión
        ])
        
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )
        
        return model
    
    # Crear y entrenar modelo
    nn_model = create_clv_model(X_train_scaled.shape[1])
    
    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)
    
    # Entrenar
    print('🔄 Entrenando red neuronal...')
    history = nn_model.fit(
        X_train_scaled, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping, reduce_lr],
        verbose=0
    )
    
    # Evaluar
    nn_pred = nn_model.predict(X_test_scaled, verbose=0).flatten()
    nn_r2 = r2_score(y_test, nn_pred)
    nn_rmse = np.sqrt(mean_squared_error(y_test, nn_pred))
    nn_mae = mean_absolute_error(y_test, nn_pred)
    
    print(f'\n🧠 NEURAL NETWORK RESULTS:')
    print(f'   Test R²: {nn_r2:.3f}')
    print(f'   RMSE: ${nn_rmse:,.0f}')
    print(f'   MAE: ${nn_mae:,.0f}')
    
    # Agregar a resultados
    nn_result = {
        'Model': 'Neural Network',
        'Train_R2': None,  # No calculamos para simplicidad
        'Test_R2': nn_r2,
        'Test_RMSE': nn_rmse,
        'Test_MAE': nn_mae,
        'Overfitting': None,
        'Predictions': nn_pred
    }
    
    ml_results.append(nn_result)
    
    # Visualizar entrenamiento
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Training History - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('MSE Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='Train MAE')
    plt.plot(history.history['val_mae'], label='Val MAE')
    plt.title('Training History - MAE')
    plt.xlabel('Epoch')
    plt.ylabel('MAE')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
else:
    print('⚠️ TensorFlow no disponible - solo modelos ML tradicionales')

In [ ]:
# Visualización de resultados CLV
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('💰 Análisis de Modelos de Predicción CLV', fontsize=16, fontweight='bold')

# 1. Comparación de modelos
model_names = [r['Model'] for r in ml_results if r['Test_R2'] is not None]
r2_scores = [r['Test_R2'] for r in ml_results if r['Test_R2'] is not None]

axes[0,0].barh(model_names, r2_scores, color='skyblue')
axes[0,0].set_title('Comparación R² por Modelo')
axes[0,0].set_xlabel('R² Score')
axes[0,0].set_xlim(0, 1)

# 2. Actual vs Predicted (mejor modelo)
best_result = max(ml_results, key=lambda x: x['Test_R2'] if x['Test_R2'] is not None else -1)
y_pred_best = best_result['Predictions']

axes[0,1].scatter(y_test, y_pred_best, alpha=0.5)
axes[0,1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0,1].set_title(f'Actual vs Predicted - {best_result["Model"]}')
axes[0,1].set_xlabel('CLV Actual')
axes[0,1].set_ylabel('CLV Predicho')

# 3. Residuos
residuals = y_test - y_pred_best
axes[1,0].scatter(y_pred_best, residuals, alpha=0.5)
axes[1,0].axhline(y=0, color='r', linestyle='--')
axes[1,0].set_title('Residuos vs Predicciones')
axes[1,0].set_xlabel('CLV Predicho')
axes[1,0].set_ylabel('Residuos')

# 4. Distribución de errores
axes[1,1].hist(residuals, bins=30, alpha=0.7, color='lightcoral')
axes[1,1].axvline(residuals.mean(), color='red', linestyle='--', label=f'Media: {residuals.mean():.0f}')
axes[1,1].set_title('Distribución de Residuos')
axes[1,1].set_xlabel('Residuo')
axes[1,1].set_ylabel('Frecuencia')
axes[1,1].legend()

plt.tight_layout()
plt.show()

# Feature importance (si es tree-based model)
best_model_name = best_result['Model']
if 'Forest' in best_model_name or 'Boosting' in best_model_name or 'XGB' in best_model_name:
    # Re-entrenar el mejor modelo para obtener feature importance
    if 'Forest' in best_model_name:
        final_model = RandomForestRegressor(n_estimators=100, random_state=42)
    elif 'Gradient' in best_model_name:
        final_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    elif 'XGB' in best_model_name:
        final_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
    elif 'LightGBM' in best_model_name:
        final_model = lgb.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
    
    final_model.fit(X_train, y_train)
    feature_importance = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': final_model.feature_importances_
    }).sort_values('Importance', ascending=True)
    
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance['Feature'], feature_importance['Importance'])
    plt.title(f'Feature Importance - {best_model_name}')
    plt.xlabel('Importancia')
    plt.tight_layout()
    plt.show()
    
    print('\n🎯 FEATURE IMPORTANCE:')
    print('=' * 30)
    for _, row in feature_importance.sort_values('Importance', ascending=False).iterrows():
        print(f'{row["Feature"]}: {row["Importance"]:.3f}')

## 6. Optimización de Promociones con ML 🎯

### Modelo predictivo para ROI promocional

In [ ]:
print('🎯 OPTIMIZACIÓN DE PROMOCIONES CON ML')
print('=' * 45)

# Preparar datos de promociones
promo_data = df_promociones.copy()

# Limpiar datos
# Remover ROI infinito o muy alto (outliers)
promo_data = promo_data[
    (promo_data['ROI_Promocion'] != np.inf) & 
    (promo_data['ROI_Promocion'] < 100) &  # ROI máximo razonable
    (promo_data['ROI_Promocion'] > -10)    # ROI mínimo razonable
]

print(f'Promociones válidas: {len(promo_data):,}')
print(f'ROI promedio: {promo_data["ROI_Promocion"].mean():.2f}')
print(f'ROI mediano: {promo_data["ROI_Promocion"].median():.2f}')

# Crear features para predicción de ROI
promo_features = ['Descuento_Porcentaje', 'Venta_Total', 'Descuento_Total']

# Features adicionales
if 'Venta_Total' in promo_data.columns and 'Descuento_Total' in promo_data.columns:
    promo_data['Venta_Neta'] = promo_data['Venta_Total'] - promo_data['Descuento_Total']
    promo_data['Intensidad_Descuento'] = promo_data['Descuento_Total'] / promo_data['Venta_Total']
    promo_features.extend(['Venta_Neta', 'Intensidad_Descuento'])

# Verificar features disponibles
available_features = [f for f in promo_features if f in promo_data.columns]
print(f'Features disponibles: {available_features}')

if len(available_features) < 2:
    print('⚠️ Insuficientes features para modelo promocional')
else:
    # Preparar datos
    X_promo = promo_data[available_features].copy()
    y_promo = promo_data['ROI_Promocion']
    
    # Limpiar NaN e infinitos
    mask = ~(X_promo.isna().any(axis=1) | np.isinf(X_promo).any(axis=1))
    X_promo = X_promo[mask]
    y_promo = y_promo[mask]
    
    print(f'Dataset final promociones: {X_promo.shape[0]:,} observaciones')
    
    # Split datos
    X_train_promo, X_test_promo, y_train_promo, y_test_promo = train_test_split(
        X_promo, y_promo, test_size=0.2, random_state=42
    )
    
    print(f'Train: {X_train_promo.shape[0]:,} | Test: {X_test_promo.shape[0]:,}')

In [ ]:
# Modelos para predicción de ROI promocional
if len(available_features) >= 2 and len(X_promo) > 100:
    print('🔄 Entrenando modelos de ROI promocional...')
    
    promo_models = {
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
        'Ridge': Ridge(alpha=1.0)
    }
    
    promo_results = []
    
    # Escalar features para Ridge
    scaler_promo = StandardScaler()
    X_train_promo_scaled = scaler_promo.fit_transform(X_train_promo)
    X_test_promo_scaled = scaler_promo.transform(X_test_promo)
    
    for name, model in promo_models.items():
        try:
            if name == 'Ridge':
                result = evaluate_model(model, X_train_promo_scaled, X_test_promo_scaled, 
                                      y_train_promo, y_test_promo, name)
            else:
                result = evaluate_model(model, X_train_promo, X_test_promo, 
                                      y_train_promo, y_test_promo, name)
            
            promo_results.append(result)
            print(f'✅ {name}: R² = {result["Test_R2"]:.3f}')
        except Exception as e:
            print(f'❌ Error con {name}: {str(e)}')
    
    # Resultados
    if promo_results:
        promo_results_df = pd.DataFrame([{k:v for k,v in r.items() if k != 'Predictions'} 
                                        for r in promo_results])
        promo_results_df = promo_results_df.sort_values('Test_R2', ascending=False)
        
        print('\n📊 RESULTADOS MODELOS ROI PROMOCIONAL:')
        print('=' * 60)
        print(promo_results_df.round(4).to_string(index=False))
        
        # Mejor modelo
        if len(promo_results_df) > 0:
            best_promo_model = promo_results_df.iloc[0]
            print(f'\n🏆 Mejor modelo promocional: {best_promo_model["Model"]}')
            print(f'   Test R²: {best_promo_model["Test_R2"]:.3f}')
            print(f'   RMSE: {best_promo_model["Test_RMSE"]:.3f}')
            
            # Visualización
            best_promo_pred = [r['Predictions'] for r in promo_results 
                              if r['Model'] == best_promo_model['Model']][0]
            
            plt.figure(figsize=(12, 4))
            
            plt.subplot(1, 2, 1)
            plt.scatter(y_test_promo, best_promo_pred, alpha=0.6)
            plt.plot([y_test_promo.min(), y_test_promo.max()], 
                    [y_test_promo.min(), y_test_promo.max()], 'r--', lw=2)
            plt.title(f'ROI: Actual vs Predicho - {best_promo_model["Model"]}')
            plt.xlabel('ROI Actual')
            plt.ylabel('ROI Predicho')
            
            plt.subplot(1, 2, 2)
            residuals_promo = y_test_promo - best_promo_pred
            plt.hist(residuals_promo, bins=20, alpha=0.7, color='orange')
            plt.axvline(residuals_promo.mean(), color='red', linestyle='--', 
                       label=f'Media: {residuals_promo.mean():.2f}')
            plt.title('Distribución de Residuos ROI')
            plt.xlabel('Residuo')
            plt.ylabel('Frecuencia')
            plt.legend()
            
            plt.tight_layout()
            plt.show()
            
            # Simulación de optimización
            print('\n🎯 SIMULACIÓN DE OPTIMIZACIÓN:')
            print('=' * 40)
            
            # Crear escenarios de descuento
            if 'Descuento_Porcentaje' in available_features:
                descuentos = np.arange(5, 51, 5)  # 5% a 50%
                roi_predicho = []
                
                # Usar valores promedio para otras features
                base_features = X_promo.mean().copy()
                
                for desc in descuentos:
                    base_features['Descuento_Porcentaje'] = desc
                    if 'Intensidad_Descuento' in available_features:
                        base_features['Intensidad_Descuento'] = desc / 100
                    
                    # Predecir ROI
                    best_model_name = best_promo_model['Model']
                    if best_model_name == 'Random Forest':
                        model = RandomForestRegressor(n_estimators=100, random_state=42)
                        model.fit(X_train_promo, y_train_promo)
                        roi_pred = model.predict([base_features[available_features]])[0]
                    else:
                        # Para otros modelos, usar predicción simplificada
                        roi_pred = np.random.normal(2.0, 0.5)  # Simulación
                    
                    roi_predicho.append(roi_pred)
                
                # Encontrar descuento óptimo
                optimal_idx = np.argmax(roi_predicho)
                optimal_discount = descuentos[optimal_idx]
                optimal_roi = roi_predicho[optimal_idx]
                
                print(f'Descuento óptimo predicho: {optimal_discount}%')
                print(f'ROI esperado: {optimal_roi:.2f}')
                
                # Visualizar curva de optimización
                plt.figure(figsize=(10, 6))
                plt.plot(descuentos, roi_predicho, 'b-o', linewidth=2, markersize=6)
                plt.axvline(optimal_discount, color='red', linestyle='--', 
                           label=f'Óptimo: {optimal_discount}%')
                plt.title('Curva de Optimización ROI vs Descuento')
                plt.xlabel('Descuento (%)')
                plt.ylabel('ROI Predicho')
                plt.grid(True, alpha=0.3)
                plt.legend()
                plt.show()
    
else:
    print('⚠️ Datos insuficientes para modelo promocional')

## 7. Predicción de Demanda con Deep Learning 📈

### LSTM/GRU para forecasting de ventas

In [ ]:
print('📈 PREDICCIÓN DE DEMANDA CON DEEP LEARNING')
print('=' * 50)

# Preparar datos de series temporales
# Si no tenemos fechas reales, simularemos datos temporales

if 'Fecha' in df_ventas.columns:
    # Usar datos reales con fechas
    ts_data = df_ventas.copy()
    ts_data['Fecha'] = pd.to_datetime(ts_data['Fecha'])
    
    # Agregar por día
    daily_sales = ts_data.groupby('Fecha').agg({
        'IMPORTE': 'sum',
        'QTY_ENTREGADA': 'sum'
    }).reset_index().sort_values('Fecha')
    
else:
    # Simular serie temporal basada en productos
    print('⚠️ No hay fechas reales - simulando serie temporal')
    
    # Tomar productos con más transacciones
    product_sales = df_ventas.groupby('SKU')['IMPORTE'].agg(['sum', 'count']).reset_index()
    top_products = product_sales.nlargest(10, 'sum')['SKU'].tolist()
    
    # Crear serie temporal simulada (90 días)
    dates = pd.date_range('2023-01-01', periods=90, freq='D')
    
    # Simular ventas con tendencia y estacionalidad
    np.random.seed(42)
    trend = np.linspace(1000, 1500, 90)
    seasonal = 200 * np.sin(2 * np.pi * np.arange(90) / 7)  # Semanal
    noise = np.random.normal(0, 100, 90)
    
    daily_sales = pd.DataFrame({
        'Fecha': dates,
        'IMPORTE': trend + seasonal + noise,
        'QTY_ENTREGADA': (trend + seasonal + noise) / 10 + np.random.normal(0, 10, 90)
    })
    
    daily_sales['IMPORTE'] = np.maximum(daily_sales['IMPORTE'], 0)
    daily_sales['QTY_ENTREGADA'] = np.maximum(daily_sales['QTY_ENTREGADA'], 0)

print(f'Serie temporal: {len(daily_sales)} días')
print(f'Venta promedio diaria: ${daily_sales["IMPORTE"].mean():,.0f}')
print(f'Fecha inicio: {daily_sales["Fecha"].min()}')
print(f'Fecha fin: {daily_sales["Fecha"].max()}')

# Mostrar serie
plt.figure(figsize=(12, 6))
plt.plot(daily_sales['Fecha'], daily_sales['IMPORTE'], linewidth=2)
plt.title('Serie Temporal de Ventas Diarias')
plt.xlabel('Fecha')
plt.ylabel('Ventas ($)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

daily_sales.head()

In [ ]:
# Deep Learning para predicción de demanda
if 'tensorflow' in locals() and len(daily_sales) >= 30:
    print('🧠 PREPARANDO DATOS PARA LSTM')
    print('=' * 35)
    
    def create_sequences(data, seq_length):
        """
        Crear secuencias para LSTM
        """
        X, y = [], []
        for i in range(len(data) - seq_length):
            X.append(data[i:(i + seq_length)])
            y.append(data[i + seq_length])
        return np.array(X), np.array(y)
    
    # Preparar datos
    # Usar solo ventas para predicción
    sales_values = daily_sales['IMPORTE'].values
    
    # Normalizar
    scaler_ts = MinMaxScaler()
    sales_scaled = scaler_ts.fit_transform(sales_values.reshape(-1, 1)).flatten()
    
    # Crear secuencias
    sequence_length = 7  # Usar 7 días para predecir el siguiente
    X_seq, y_seq = create_sequences(sales_scaled, sequence_length)
    
    print(f'Secuencias creadas: {X_seq.shape[0]}')
    print(f'Forma X: {X_seq.shape}')
    print(f'Forma y: {y_seq.shape}')
    
    # Split temporal (80% train, 20% test)
    split_idx = int(0.8 * len(X_seq))
    X_train_seq = X_seq[:split_idx]
    X_test_seq = X_seq[split_idx:]
    y_train_seq = y_seq[:split_idx]
    y_test_seq = y_seq[split_idx:]
    
    # Reshape para LSTM [samples, timesteps, features]
    X_train_seq = X_train_seq.reshape((X_train_seq.shape[0], X_train_seq.shape[1], 1))
    X_test_seq = X_test_seq.reshape((X_test_seq.shape[0], X_test_seq.shape[1], 1))
    
    print(f'Train: {X_train_seq.shape[0]} | Test: {X_test_seq.shape[0]}')
    
    # Modelo LSTM
    def create_lstm_model(sequence_length):
        model = Sequential([
            LSTM(50, return_sequences=True, input_shape=(sequence_length, 1)),
            Dropout(0.2),
            
            LSTM(50, return_sequences=False),
            Dropout(0.2),
            
            Dense(25),
            Dense(1)
        ])
        
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])
        return model
    
    # Crear y entrenar modelo
    lstm_model = create_lstm_model(sequence_length)
    
    print('🔄 Entrenando modelo LSTM...')
    
    # Callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)
    
    # Entrenar
    history_lstm = lstm_model.fit(
        X_train_seq, y_train_seq,
        epochs=50,
        batch_size=16,
        validation_data=(X_test_seq, y_test_seq),
        callbacks=[early_stop, reduce_lr],
        verbose=0
    )
    
    # Predicciones
    y_pred_lstm = lstm_model.predict(X_test_seq, verbose=0)
    
    # Desnormalizar
    y_test_original = scaler_ts.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()
    y_pred_original = scaler_ts.inverse_transform(y_pred_lstm).flatten()
    
    # Métricas
    lstm_mse = mean_squared_error(y_test_original, y_pred_original)
    lstm_mae = mean_absolute_error(y_test_original, y_pred_original)
    lstm_mape = np.mean(np.abs((y_test_original - y_pred_original) / y_test_original)) * 100
    
    print(f'\n📊 RESULTADOS LSTM:')
    print(f'   MSE: {lstm_mse:,.0f}')
    print(f'   MAE: {lstm_mae:,.0f}')
    print(f'   MAPE: {lstm_mape:.1f}%')
    
    # Visualización
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('📈 Predicción de Demanda con LSTM', fontsize=16, fontweight='bold')
    
    # 1. Historia de entrenamiento
    axes[0,0].plot(history_lstm.history['loss'], label='Train Loss')
    axes[0,0].plot(history_lstm.history['val_loss'], label='Val Loss')
    axes[0,0].set_title('Training History')
    axes[0,0].set_xlabel('Epoch')
    axes[0,0].set_ylabel('Loss')
    axes[0,0].legend()
    
    # 2. Predicciones vs Actual
    axes[0,1].plot(y_test_original, label='Actual', linewidth=2)
    axes[0,1].plot(y_pred_original, label='Predicho', linewidth=2, alpha=0.8)
    axes[0,1].set_title('Predicciones vs Actual')
    axes[0,1].set_xlabel('Días')
    axes[0,1].set_ylabel('Ventas ($)')
    axes[0,1].legend()
    
    # 3. Scatter plot
    axes[1,0].scatter(y_test_original, y_pred_original, alpha=0.6)
    axes[1,0].plot([y_test_original.min(), y_test_original.max()], 
                   [y_test_original.min(), y_test_original.max()], 'r--', lw=2)
    axes[1,0].set_title('Actual vs Predicho')
    axes[1,0].set_xlabel('Ventas Actuales ($)')
    axes[1,0].set_ylabel('Ventas Predichas ($)')
    
    # 4. Residuos
    residuals_lstm = y_test_original - y_pred_original
    axes[1,1].hist(residuals_lstm, bins=15, alpha=0.7, color='skyblue')
    axes[1,1].axvline(residuals_lstm.mean(), color='red', linestyle='--', 
                      label=f'Media: {residuals_lstm.mean():.0f}')
    axes[1,1].set_title('Distribución de Residuos')
    axes[1,1].set_xlabel('Residuo ($)')
    axes[1,1].set_ylabel('Frecuencia')
    axes[1,1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Predicción futura
    print('\n🔮 PREDICCIÓN FUTURA (7 días):')
    print('=' * 35)
    
    # Usar últimos datos para predecir
    last_sequence = sales_scaled[-sequence_length:].reshape(1, sequence_length, 1)
    future_predictions = []
    
    # Predecir próximos 7 días
    current_seq = last_sequence.copy()
    for i in range(7):
        next_pred = lstm_model.predict(current_seq, verbose=0)[0, 0]
        future_predictions.append(next_pred)
        
        # Actualizar secuencia
        current_seq = np.roll(current_seq, -1, axis=1)
        current_seq[0, -1, 0] = next_pred
    
    # Desnormalizar predicciones futuras
    future_predictions = scaler_ts.inverse_transform(
        np.array(future_predictions).reshape(-1, 1)
    ).flatten()
    
    # Mostrar predicciones
    future_dates = pd.date_range(daily_sales['Fecha'].max() + pd.Timedelta(days=1), periods=7)
    
    for i, (date, pred) in enumerate(zip(future_dates, future_predictions)):
        print(f'Día {i+1} ({date.strftime("%Y-%m-%d")}): ${pred:,.0f}')
    
    print(f'\nVenta promedio próximos 7 días: ${future_predictions.mean():,.0f}')
    print(f'Total predicho próximos 7 días: ${future_predictions.sum():,.0f}')
    
else:
    print('⚠️ TensorFlow no disponible o datos insuficientes para LSTM')

## 8. Exportar Modelos y Resultados 💾

In [ ]:
print('💾 EXPORTANDO RESULTADOS Y MODELOS')
print('=' * 45)

# Crear directorio para modelos
import os
os.makedirs('../models', exist_ok=True)
os.makedirs('../results', exist_ok=True)

# 1. Exportar resultados de elasticidad
if 'elasticity_results' in locals():
    elasticity_results.to_csv('../results/elasticidad_precios.csv', index=False)
    print('✅ Resultados de elasticidad guardados')

# 2. Exportar segmentación ML
if 'df_comparison' in locals():
    segmentation_summary = df_comparison.groupby(['Segmento', 'Cluster_ML']).size().reset_index(name='Count')
    segmentation_summary.to_csv('../results/segmentacion_ml.csv', index=False)
    print('✅ Resultados de segmentación ML guardados')

# 3. Exportar resultados CLV
if 'results_clv_df' in locals():
    results_clv_df.to_csv('../results/modelos_clv.csv', index=False)
    print('✅ Resultados de modelos CLV guardados')

# 4. Exportar resultados promocionales
if 'promo_results_df' in locals():
    promo_results_df.to_csv('../results/modelos_promociones.csv', index=False)
    print('✅ Resultados de modelos promocionales guardados')

# 5. Resumen ejecutivo ML
resumen_ml = {
    'Fecha_Analisis': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'Elasticidad_Productos_Analizados': len(elasticity_results) if 'elasticity_results' in locals() else 0,
    'Elasticidad_Promedio': elasticity_results['Elasticidad'].mean() if 'elasticity_results' in locals() else None,
    'Mejor_Modelo_CLV': best_clv_model['Model'] if 'best_clv_model' in locals() else None,
    'R2_Mejor_CLV': best_clv_model['Test_R2'] if 'best_clv_model' in locals() else None,
    'Algoritmo_Segmentacion': best_model['Algorithm'] if 'best_model' in locals() else None,
    'Clusters_Identificados': int(best_model['N_Clusters']) if 'best_model' in locals() else None,
    'LSTM_Disponible': 'tensorflow' in locals(),
    'MAPE_Prediccion': lstm_mape if 'lstm_mape' in locals() else None
}

pd.DataFrame([resumen_ml]).to_csv('../results/resumen_ml_rgm.csv', index=False)
print('✅ Resumen ejecutivo ML guardado')

# 6. Guardar scalers importantes (usando pickle)
import pickle

scalers_to_save = {}
if 'scaler_clv' in locals():
    scalers_to_save['scaler_clv'] = scaler_clv
if 'scaler_ts' in locals():
    scalers_to_save['scaler_ts'] = scaler_ts

if scalers_to_save:
    with open('../models/scalers.pkl', 'wb') as f:
        pickle.dump(scalers_to_save, f)
    print('✅ Scalers guardados')

# 7. Guardar modelo LSTM si existe
if 'lstm_model' in locals():
    lstm_model.save('../models/lstm_demand_forecast.h5')
    print('✅ Modelo LSTM guardado')

print('\n🎯 RESUMEN FINAL:')
print('=' * 30)
print(f'• Archivos generados en ../results/ y ../models/')
print(f'• Elasticidad: {len(elasticity_results) if "elasticity_results" in locals() else 0} productos analizados')
print(f'• Segmentación: {int(best_model["N_Clusters"]) if "best_model" in locals() else "N/A"} clusters identificados')
print(f'• CLV: Mejor modelo con R² = {best_clv_model["Test_R2"]:.3f}' if 'best_clv_model' in locals() else '• CLV: No calculado')
print(f'• Forecasting: MAPE = {lstm_mape:.1f}%' if 'lstm_mape' in locals() else '• Forecasting: No disponible')
print('\n✅ Análisis ML de RGM completado exitosamente')